In [1]:
import ape
from ape.api import ProviderAPI, BlockAPI, TransactionAPI
from ape import Contract
from ape.contracts import ContractInstance
from ape.contracts.base import ContractTransaction, ContractTransactionHandler

/home/mihai/.pyenv/versions/3.9.16/envs/elf-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 ## Set up ape

In [2]:
USE_ALCHEMY = True
PROVIDER_STRING = "alchemy" if USE_ALCHEMY else "http://localhost:8547"
provider: ProviderAPI = ape.networks.parse_network_choice(f"ethereum:goerli:{PROVIDER_STRING}").push_provider()

 ### show original `block.base_fee` behavior

In [3]:
block = ape.chain.blocks[-1]
block2 = provider.get_block("latest")
block3 = provider.get_block("pending")
assert block == block2
print(f"block.base_fee  = {getattr(block, 'base_fee'):,.0f} in {block=}")
print(f"block2.base_fee = {getattr(block2, 'base_fee'):,.0f} in {block2=}")
print(f"block3.base_fee = {getattr(block3, 'base_fee'):,.0f} in {block3=}")

ValidationError: 1 validation error for Block
totalDifficulty
  none is not an allowed value (type=type_error.none.not_allowed)

 ### show old `provider.base_fee` behavior

In [4]:
fee_history = provider.web3.eth.fee_history(1, provider.web3.eth.get_block("latest").number)
last_base_fee, next_base_fee = fee_history["baseFeePerGas"]
print(f"current base fee is         {last_base_fee/1e9:,.2f} 😴")
print(f"base fee FROM THE FUTURE is {next_base_fee/1e9:,.2f} 🤪")
print(f"provider's base fee  is...  {getattr(provider, 'base_fee')/1e9:,.2f} 🤦")

current base fee is         700.46 😴
base fee FROM THE FUTURE is 671.69 🤪
provider's base fee  is...  700.46 🤦


### preparing a new transaction uses the current (latest block) base fee

In [5]:
dai: ContractInstance = Contract("0x11fe4b6ae13d2a6055c8d9cf65c55bac32b5d844")  # goerli DAI
contract_txn = ContractTransaction(address=dai.address,abi=dai.approve.abis[0])
serial_txn: TransactionAPI = contract_txn.serialize_transaction("0x" + "0"*40, 1, gas_limit=int(1e6))
prepped_txn: TransactionAPI = provider.prepare_transaction(serial_txn)
max_fee, max_priority_fee = prepped_txn.max_fee, prepped_txn.max_priority_fee
implied_base_fee = max_fee - max_priority_fee
print(f"base fee = {implied_base_fee/1e9:,.2f}")
print(f"         = max_fee ({max_fee/1e9:,.2f}) - max_priority_fee ({max_priority_fee/1e9:,.2f})")
if implied_base_fee == last_base_fee:
    print(f"this matches LAST base fee {last_base_fee/1e9:,.2f}")
elif implied_base_fee == next_base_fee:
    print(f"this matches NEXT base fee {next_base_fee/1e9:,.2f}")

base fee = 700.46
         = max_fee (701.87) - max_priority_fee (1.41)
this matches LAST base fee 700.46
